In [1]:
# %%writefile datafusion_pipev0.py
# A simple pipeline structure that can run different steps in a compute cluster
import azure.core
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails

from azureml.data import OutputFileDatasetConfig
from vdsm_util import create_blob_stream
import json
ws = Workspace.from_config()
# print(ws)
blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'vdsm_fusion_test1'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'vdsm-cluster-d13'  # ALL SMALL LETTER, NO underscore, 16ws long only.
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :", aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D13",    #56G Ram, 8 cores
                                                                min_nodes=0, 
                                                                max_nodes=8,
                                               idle_seconds_before_scaledown=300)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
print("Azure Machine Learning Compute Cluster attached now")

# Create new environment to execute your code


# env = Environment.from_conda_specification(name=env_name, file_path="conda_dependencies.yml")
# env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
# env.register(workspace=ws)

# Or use an existing env which was registered to the ws
env_name = "vdsm_env_all_test3" 
rebuild_env = False
if rebuild_env:
    env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
    env.register(workspace=ws)
else:
    envs = Environment.list(workspace=ws)
    if env_name in envs:
        # Use an existing environment
        env = Environment.get(workspace=ws, name=env_name)
    else:
        # Create a new env using yml
        env = Environment.from_conda_specification(name=env_name, file_path="vdsm_all.yml")
        # Register the environment 
        env.register(workspace=ws)

# ### Here are some examples about how to add packages to an existing env.
# conda_dep = CondaDependencies()
# # Installs numpy version 1.17.0 conda package
# conda_dep.add_conda_package("numpy==1.17.0")
# # Installs pillow package
# conda_dep.add_pip_package("pillow")
# # Adds dependencies to PythonSection of myenv
# env.python.conda_dependencies=conda_dep

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = aml_compute

# Assign the environment to the run configuration
pipeline_run_config.environment = env
pipeline_run_config.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"




This Compute Target already exist.
Azure Machine Learning Compute Cluster attached now


In [12]:
# print(type(s2_path))
print ("Run configuration created.")
## Pipeline blueprint: test data fusion first
### To-add: data prep script from Catriona
with open('cred_blob_ps.json') as psfile:
    cred = json.load(psfile)

# s2_path = create_blob_stream('odc_vm_data/odc_outputs/W_angelas_2_evibands_2015-12_2021-10_test.pkl', t_expire=1, cred=cred)
# fc_path = create_blob_stream('odc_vm_data/odc_outputs/W_angelas_fc_annual_cover_15_20_test.pkl', t_expire=1, cred=cred)
# Example that works in the original code
# co2_path = create_blob_stream('odc_vm_data/odc_outputs/co2_mm_gl_url.txt', t_expire=8)
# co2_path = create_blob_stream('odc_vm_data/odc_outputs/co2_mm_gl_url.txt', t_expire=1, cred=cred)
# #  Create a url stream for the monthly climate data
# # climate_path = create_blob_stream('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc', t_expire=8)
climate_path = create_blob_stream('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc', t_expire=1, cred=cred)
# mask_path = create_blob_stream('odc_vm_data/odc_outputs/W_angelas_all7poly.geojson', t_expire=1, cred=cred)

with open('cred_blob_af.json') as psfile:
    cred = json.load(psfile)

s2_path = create_blob_stream('tmp/YA_308_311_S2_2015-12_2021-07.pkl', t_expire=8, cred=cred)
fc_path = create_blob_stream('tmp/YA_308_311_fc_2015_2020.pkl', t_expire=8, cred=cred)

co2_path = create_blob_stream('co2/co2_mm_gl_url.txt', t_expire=1, cred=cred)
# climate_path = create_blob_stream('/home/azureuser/cloudfiles/code/Users/Tiantian.Yuan/test_only/era-mon-2015-2021.nc', t_expire=1, cred=cred)
mask_path = create_blob_stream('static/RTiO_ChainageAreas_1KM_30M.json', t_expire=1, cred=cred)
# check if path correctly assigned
# print(s2_path)


Run configuration created.


In [13]:

fusion_step = PythonScriptStep(
    name="veg_data_fusion",
    script_name="vdsm_traindata_fusion_single.py",
    compute_target=aml_compute_target,
    arguments=[
        "--s2-path", s2_path,
        "--fc-path", fc_path,
        "--co2-path", co2_path,
        "--climate-path", climate_path,
        "--mask-path", mask_path,
        "--random-n", 50,
        "--PV", False,
        "--NPV", True,
        "--fused-data-train-folder", "fused_data_train_npv",
        "--fused-data-train-filename", "fusetrain_data_npv_5km.csv"
    ],
    source_directory=".", 
    runconfig=pipeline_run_config,
    allow_reuse=True)

### Catriona task: concat the output from the last step from different areas and produce a master tabular data. 
###   possible add registering the master dataset to our ML DevOps pipeline step.
# concat_step = PythonScriptStep(
#     name="VDSM_concat",
#     script_name="VDSM_concat.py",
#     compute_target=aml_compute_target,
#     arguments=["--prepped_data", prepped_data],
#     source_directory="prep_src"
# )

# steps = [dataprep_step, fusion_step, concat_step]
steps = [fusion_step]
pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = exp.submit(pipeline)
print('VDSM training data fusion submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()


Created step veg_data_fusion [1a0d6b08][3a3cc4c3-38bd-4ff8-98bf-f3ecd6448381], (This step will run and generate new outputs)
Submitted PipelineRun 749b1d8d-3049-4f42-959b-b169e2d01146
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/749b1d8d-3049-4f42-959b-b169e2d01146?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
VDSM training data fusion submitted for execution
PipelineRunId: 749b1d8d-3049-4f42-959b-b169e2d01146
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/749b1d8d-3049-4f42-959b-b169e2d01146?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRun Status: Running


StepRunId: 07d35da0-5c46-4410-96dd-3673a2ebf6d2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/07d35da0-5c46-4410-96dd-3673a2ebf6d2?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( veg_data_fusion ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_5655295f115745520bbc4ed391332fea10f7875ea0fbb6a572f2af3cc72fb902_d.txt
2021-12-05T09:00:16Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/07d35da0-5c46-4410-96dd-3673a2ebf6d2/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/aml-ws-spacetech001/azureml/07d35da0-5c46-4410-96dd-3673a2ebf6d2/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=369434 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jo

In [3]:
# !ipython kernel install --user --name vdsm_env_all_test0 --display-name "vdsm_env_all_test0"

Installed kernelspec vdsm_env_all_test0 in /home/azureuser/.local/share/jupyter/kernels/vdsm_env_all_test0


In [2]:
import xarray as xr
import geopandas as gpd